# DISEÑO DE UNA RED NEURONAL

##  Edison David González Blandón - JuanEsteban Velásquez

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df1=pd.read_csv("DataNN.txt")
df2 = df1.copy()
classes = df1.iloc[:,-1].copy()
df1 = df1.iloc[:,0:-1].copy()
df1 = (df1-np.mean(df1, axis = 0))/(np.std(df1, axis = 0))
NumVariables = df1.shape[1]
X = np.array(df1)
y = np.array(classes)-1.0


In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def d_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

def clearList(L): #arroja una lista de L elementos vacios, no sabia de qué otra forma hacerla
    listica = []
    for i in range(0,L):
        listica.append([]);
    return listica

def RedNeuronalTrain(X,y,Lsizes,alpha = .1,epochs = 20,outputs = 2):
    N  = X.shape[0]   #número de muestras para realizar el entrenamiento
    nf = X.shape[1]   #número de características
    L = len(Lsizes)   #número de capas ocultas
    w_ = []           #pesos de la red neuronal
    error_array = []
    w_.append(np.random.rand(Lsizes[0],nf+1)) #Primera capa: L_1 filas & nf + 1 columnas (añadiendo la neurona de BIAS)
    for k in range(0,L-1):
        w_.append(np.random.rand(Lsizes[k+1],(Lsizes[k]+1)))#capa siguiente en las filas y capa actual columnas
    w_.append(np.random.rand(outputs,Lsizes[L-1]+1))        #Ultima capa interna: ourput_N flas & L_final columnas 
                                                            #(añadiendo la neurona de BIAS) 
    for j in range(0,epochs): #condición de parada
        J = 0                 #error cuadratico medio  
        for i in range(0,N):
            Y = np.zeros(shape=(outputs,1))
            Y[int(y[i])] = 1.0
            a_ = []
            z_ = []
            a_.append(X[i,None].T)
            for k in range(0,(L+1)):
                #Se agrega el bias a la capa externa k+1
                a_[k] = np.insert(a_[k], 0, 1,axis=0)
                z_.append(np.dot(w_[k],a_[k]))
                a_.append(sigmoid(z_[k]))
            #Backward
            J = J + np.sum((Y-a_[len(a_)-1])*(Y-a_[len(a_)-1]))
            d_=clearList(len(a_))
            
            d_[L+1]= (Y-a_[L+1])*d_sigmoid(z_[L])
            for k in range(L,0,-1):
                d_[k] = np.dot(w_[k].T,d_[k+1])
                d_[k] = np.delete(d_[k],0,axis=0)
                d_[k] = d_[k]*d_sigmoid(z_[k-1])
                
            DE_dw_=clearList(len(w_))
            for k in range(0,len(w_)):
                DE_dw_[k] = -np.dot(d_[k+1],a_[k].T)
            for k in range(0,len(w_)):
                w_[k] = w_[k]-alpha*DE_dw_[k]
        error_array.append(J/2)
    return [w_,J/2,error_array]

def Predict(X,w_): 
    #Forward
    #Para cada muestra
    L = len(w_)
    a_ = []
    z_ = []
    a_.append(X[:,None])
    for k in range(0,(L)):
        #Se agrega el bias a la capa externa k+1
        a_[k] = np.insert(a_[k], 0, 1,axis=0)
        z_.append(np.dot(w_[k],a_[k]))
        a_.append(sigmoid(z_[k]))
    return a_[len(a_)-1]
